In [11]:
# Main notebook for comparing new data with SSI figures from modelling reports
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
# plt.rcParams["image.cmap"] = "Dark2"
# plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.Dark2.colors)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
%matplotlib widget
# plt.style.use('ggplot')
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))
from matplotlib import cm # Colormaps

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax = plt.gca()
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# # ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y \n %B'))

import os
# import csv
import math


from datetime import date


saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]
    
# Define paths
rootdir_data = os.getcwd() +"\\..\\DanskeData\\" 

path_data = rootdir_data + "ssi_data\\"
path_dash = rootdir_data + "ssi_dashboard\\"
path_vacc = rootdir_data + "ssi_vacc\\"

path_figs = os.getcwd() +"\\..\\Figures\\" 

saveFigures is set to: True
Done loading packages


In [12]:

latestsubdir = list(os.walk(path_data))[0][1][-1]
latestdir = path_data + latestsubdir

dfCase = pd.read_csv(latestdir+'/Test_pos_over_time.csv',delimiter = ';',dtype=str)
dfCase = dfCase.iloc[:-2]
dfCase['NewPositive'] = pd.to_numeric(dfCase['NewPositive'].astype(str).apply(lambda x: x.replace('.','')))
dfCase['Tested'] = pd.to_numeric(dfCase['Tested'].astype(str).apply(lambda x: x.replace('.','')))
dfCase['PosPct'] = pd.to_numeric(dfCase['PosPct'].astype(str).apply(lambda x: x.replace(',','.')))
dfCase['Date'] =  pd.to_datetime(dfCase.Date,format='%Y-%m-%d')
testDates = dfCase['Date']

dfAdm = pd.read_csv(latestdir+'/Newly_admitted_over_time.csv',delimiter = ';',dtype=str)
dfAdm['Dato'] = pd.to_datetime(dfAdm['Dato'])
dfAdm['Total'] = pd.to_numeric(dfAdm['Total'])
dfAdm.tail()


# dfDea = pd.read_csv(latestdir+'/Deaths_over_time.csv',delimiter = ';',dtype=str)
# dfDea = dfDea.iloc[:-1,:]
# dfDea['Dato'] = pd.to_datetime(dfDea['Dato'])
# dfDea['Antal_døde'] = pd.to_numeric(dfDea['Antal_døde'])
# dfDea.tail()

,Dato,Hovedstaden,Sjælland,Syddanmark,Midtjylland,Nordjylland,Ukendt Region,Total
627,2021-11-18,32,16,17,5,5,1,76
628,2021-11-19,23,24,14,6,9,1,77
629,2021-11-20,41,11,12,13,3,2,82
630,2021-11-21,35,8,12,4,7,1,67
631,2021-11-22,37,12,5,11,12,2,79


In [13]:

fig,ax1 = plt.subplots(figsize=(10,10))


import matplotlib.image as mpimg
img = mpimg.imread( os.getcwd()+'\\..\\SSI_figures\\Smittetal_Model_2021-11-01.png')
img = np.flipud(img)
imgplot = plt.imshow(img,origin='lower')


# Manual readoff of values
# y0 = 80 # 0
# y1 = 280 # 1000
# x0 = 10 
# xEnd = 122
# divRatio = (y1-y0)/1000
# xMult = (xEnd-x0)/30

y0 = 80 
y1 = 280 
x0 = 7
xEnd = 119
divRatio = (y1-y0)/1415
xMult = (xEnd-x0)/30

firstDate = np.datetime64('2021-09-02')
lastDate = np.datetime64('2021-11-22')

indexToUse = dfCase.Date >= firstDate
dataToUse = dfCase.NewPositive[indexToUse].values

# xFinal = 290
# xToUse = np.linspace(x0,xEnd,len(dataToUse))
xToUse =np.arange(0,len(dataToUse))
xToShow = xMult*xToUse+x0


dataToPlot = (divRatio*dataToUse) + y0

meanWidth = 7
# ax1.plot(xToShow,dataToPlot,'m+',markersize=10,label='Data')
ax1.plot(xToShow[:-1],dataToPlot[:-1],'m+',markersize=10,label='Data')

# ax1.plot(xToShow[-1]+(xToShow[-1]-xToShow[-2]),divRatio*last24Hours+y0,'m*',label='Seneste 24 timer\n(Ikke nøjagtigt)')
# ax1.plot(rnTime(xToShow,meanWidth),rnMean(dataToPlot,meanWidth),'k',linewidth=5,label=f'{meanWidth} dages gennemsnit')
ax1.plot(rnTime(xToShow[:-1],meanWidth),rnMean(dataToPlot[:-1],meanWidth),'k',linewidth=5,label=f'{meanWidth} dages gennemsnit')

ax1.set_xticks([])
ax1.set_yticks([])

yTicksToUse = np.arange(0,5100,500)
yTicksToShow = divRatio*yTicksToUse + y0
ax1.set_yticks(yTicksToShow)
ax1.set_yticklabels(yTicksToUse)

ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.spines['bottom'].set_visible(False)

# numDays = int((lastDate-firstDate)/np.timedelta64(1,'D'))
# xTicksToUse = np.arange(0,numDays,14) 


# ax1.legend(loc='center left')
ax1.legend(loc='upper left',bbox_to_anchor=(0,0.9,0,0))

ax1.set_ylabel('Antal nye smittetilfælde om dagen')

plt.tight_layout()

if saveFigures:
    plt.savefig(path_figs+'SSI_prediction_comparisonWithNewData')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
fig,ax1 = plt.subplots()

# ax1.plot(dfCase.Date,np.log(dfCase.NewPositive),'k.',markersize=3)
# ax1.plot(rnTime(dfCase.Date,7),rnMean(np.log(dfCase.NewPositive),7),'k')
ax1.plot(dfCase.Date,dfCase.NewPositive,'k.',markersize=3)
ax1.plot(rnTime(dfCase.Date,7),rnMean(dfCase.NewPositive,7),'k')

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))


ax1.set_yscale('log')

ax1.set_xlim([np.datetime64('2021-06-01'),np.datetime64('2021-12-01')])
ax1.set_ylim([100,10000])
ax1.grid()
# ax1.set_ylim([4,10])

ax1.set_ylabel('Antal nye smittetilfælde [logaritmisk akse]')

plt.savefig(path_figs+'CasesLogaritmisk')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Nyindlæggelser

In [15]:

fig,ax1 = plt.subplots(figsize=(10,10))


import matplotlib.image as mpimg
img = mpimg.imread( os.getcwd()+'\\..\\SSI_figures\\Indlagte_Model_2021-11-01.png')

# plt.figure()
img = np.flipud(img)
imgplot = plt.imshow(img,origin='lower')


# Manual readoff of values
# y0 = 60 # 0
# y1 = 330 # 1000
# x0 = 77
# xEnd = 166
# divRatio = (y1-y0)/60
# xMult = (xEnd-x0)/30
# y1 = 337

y0 = 83
y1 = 435
x0 = 8
xEnd = 38.15
divRatio = (y1-y0)/y0
xMult = (xEnd-x0)/x0


firstDate = np.datetime64('2021-09-02')
lastDate = np.datetime64('2021-11-22')

indexToUse = dfAdm.Dato >= firstDate
# dataToUse = dfAdm.Total[indexToUse].values
dataToUse = dfAdm.Total[indexToUse].values[:-1]

# xFinal = 290
# xToUse = np.linspace(x0,xEnd,len(dataToUse))
xToUse =np.arange(0,len(dataToUse))
xToShow = xMult*xToUse+x0


dataToPlot = (divRatio*dataToUse) + y0

meanWidth = 7
ax1.plot(xToShow,dataToPlot,'m+',markersize=10,label='Data')

# ax1.plot(xToShow[-1]+(xToShow[-1]-xToShow[-2]),divRatio*last24Hours+y0,'m*',label='Seneste 24 timer\n(Ikke nøjagtigt)')
ax1.plot(rnTime(xToShow,meanWidth),rnMean(dataToPlot,meanWidth),'k',linewidth=5,label=f'{meanWidth} dages gennemsnit')

ax1.set_xticks([])
# ax1.set_yticks([])

yTicksToUse = np.arange(0,200,25)
yTicksToShow = divRatio*yTicksToUse + y0
ax1.set_yticks(yTicksToShow)
ax1.set_yticklabels(yTicksToUse)


ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.spines['bottom'].set_visible(False)

# numDays = int((lastDate-firstDate)/np.timedelta64(1,'D'))
# xTicksToUse = np.arange(0,numDays,14) 


# ax1.legend(loc='center left')
ax1.legend(loc='upper left',bbox_to_anchor=(0.025,0.9,0,0))

ax1.set_ylabel('Antal nyindlæggelser om dagen')

plt.tight_layout()

if saveFigures:
    plt.savefig(path_figs+'SSI_prediction_comparisonWithNewData_admissions')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …